# <strong>Cross-Site Scripting (XSS)</strong>

Cross-Site Scripting (XSS) is an injection attack where hackers deliver a JavaScript payload to execute malicious code whenever someone visits a website. These types of attacks are common on websites that have message boards or comments. There are three types of XSS attacks.

<figure><center><img src="resources/xss/xss_example.jpg" style="width: 75%; height: 75%;"></img></center><em><figcaption>Credit: <a href="https://spanning.com/blog/cross-site-scripting-web-based-application-security-part-3/">Spanning</a></figcaption></em></figure>

<strong>Reflected XSS</strong> is where JavaScript is injected through an HTTP request. A website that uses CGI (Common Gateway Interface) retrieves parameters through the address bar of a browser. A reflected XSS attack injects HTML or JavaScript into an HTTP request, then tampers with the page that gets rendered to you.

Here is an example from <a href="https://portswigger.net/web-security/cross-site-scripting">PortSwigger</a>: 

```https://insecure-website.com/status?message=<script>/*+Bad+stuff+here...+*/</script>```

When the site is accessed, it will render: ```<p>Status: <script>/* Bad stuff here... */</script></p>```

The ```<script>``` tag will execute any JavaScript code that the hacker wishes to write.

<strong>Stored XSS</strong> is an attack where injected JavaScript is stored on a website by an attacker. For this lab, you will be working with stored XSS payloads. When message boards don't sanitize a user's input, a hacker can write an attack containing ```<script>```, which gets executed whenever that webpage is visited by anyone.

<strong>DOM-Based XSS</strong> is an attack similar to reflected XSS attacks, but does not utilize HTTP requests when delivering the payload. Rather than using CGI parameters, a DOM-Based attack does not send data to the server, and the attack is entirely client-side. These attacks are difficult to differentiate from reflected XSS attacks. 

<strong>This lab will contain four topics, and you will learn the following:</strong>

1. HTTP Protocols
2. Session Hijacking
3. PHP Sessions and ```document.cookie```
4. A Large-Scale Application of Cross-Site Scripting

In [1]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re
import shlex

# These are pre-set so that the first three steps will work properly, as they all
# depend on each other. Remaining booleans will be set in the cells.
step1Complete = step2Complete = step3Complete = False

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/save.py xss"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save(question, response, answer=""):
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()
    
    if answer == "":
        inner_cmd = f"/home/.education/grader.py X {question} {response}"
    else:
        inner_cmd = f"/home/.education/grader.py X {question} {response} {answer}"

    full_cmd = f"su - USERNAME_GOES_HERE -c {shlex.quote(inner_cmd)}"
    result = subprocess.run(full_cmd, shell=True, capture_output=True, text=True)

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

# This variable is used to warn students if they haven't loaded a save yet.
def warn_student():
    if (os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/.xss_warning")):
        os.remove("/project/USERNAME_GOES_HERE/notebooks/saves/.xss_warning")
        return True

def load_notebook():
    with load_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/load.py xss"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

### Step 0: Starting the Lab

Click the button to begin creating the experiment.

<strong>Note:</strong> If your buttons are not displaying, click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to render all widgets.

In [2]:
# Click the button below to start the experiment.
import time

def check_autosave():
    # Check if the student has an autosave.
    if os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/USERNAME_GOES_HERE_xss.tar.gz"):
        # Create a hidden file that will serve as a "boolean" for later.
        subprocess.run("touch /project/USERNAME_GOES_HERE/notebooks/saves/.xss_warning", shell=True)

def startlab(button):
    # Defining the lab name.
    labname = "xss"

    # Writing the information to an empty field below the button.
    with output0:
        # Fixes a strange error that happens only occasionally.
        os.chdir("/project/USERNAME_GOES_HERE/notebooks")
        output0.clear_output()

        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + "jup.USERNAME_GOES_HERE"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - USERNAME_GOES_HERE -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing activation for this lab already exists. </span><span>You might have run another \
            lab without stopping this one. Attaching the existing activation...</span>"))

            # Detaching it and re-attaching.
            subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc detach xdc.USERNAME_GOES_HERE"', shell=True, check=True)
            subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc attach xdc.USERNAME_GOES_HERE real.' + labname + 'jup.USERNAME_GOES_HERE"', capture_output=True, text=True, shell=True, check=True)
            display(HTML("<span>Re-running the installation... </span><span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

            # Check if the student has an autosave.
            check_autosave()
            
            # Re-running the installation script.
            subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/runlab ' + labname + 'jup"', capture_output=True, text=True, shell=True, check=True)
            output0.clear_output()
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been re-installed. </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing activations are found.</span>"))

            # Second, start the lab.
            display(HTML("<span>Starting the " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            try:
                startexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/startexp ' + labname + 'jup"', capture_output=True, text=True, shell=True, check=True)
            except:
                output0.clear_output()
                display(HTML("<span style='color: red;'>There was an error starting your experiment. Make sure your password is written inside of ~/pass.txt, and try again.</span>"))
                return
            
            print(startexp.stdout, flush=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout, flush=True)

            # Another lab is already attached to the XDC.
            if ("XDC already attached" in startexp.stdout):
                existingLab = re.search(r"real.(.*).USERNAME_GOES_HERE", startexp.stdout).group(1)

                # Shouldn't happen.
                if labname == existingLab:
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc detach xdc.USERNAME_GOES_HERE"', shell=True, check=True)
                    display(HTML("<span>Attaching the current lab.</span>"))
                    subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc attach xdc ' + materialPattern + '"', shell=True, check=True)
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))

            # Gives the notebook a couple seconds so that it will recognize the node(s).
            time.sleep(2)

            # Move the resources over.
            runlab = subprocess.run(
                'su - USERNAME_GOES_HERE -c "bash /home/runlab ' + labname + 'jup"',
                capture_output=True, text=True, shell=True
            )

            # Check if the student has an autosave.
            check_autosave()

            # Complete. Inform the student.
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))


# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

<hr>

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/USERNAME_GOES_HERE_xss.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 0):
                output0_2.clear_output()
                display(HTML("<span style='color: green;'>Your lab has been successfully loaded. Please click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to reflect your changes.</span>"))
            elif (result.returncode == 2):
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>The server node is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

## <strong>Topic 1: HTTP Protocols</strong>

#### <strong>HTTP Protocols</strong>

An HTTP (Hypertext Transfer Protocol) request is a method of sending information across website pages. We are going to look at ```POST``` and ```GET``` requests for this lab.

A <strong>```POST```</strong> request consists of sending data to a website, to which it becomes processed by the server. ```POST``` should not be confused with ```PUT```, where ```PUT``` is mainly used for replacing or updating a certain resource on a server. ```PUT``` requires a specific Uniform Resource Identifier (URI), meaning you must specify the exact path and resource that you want to update.

The most common use of a ```POST``` request is submitting a form on a website. The website identifies specific names for each field, like name, email, or an "accept" checkbox. Then, when the user ```POST```s the information by clicking submit, the server assigns those names with specific values that the user provided on the form, then processes the information, like adding the values to the database.

A <strong>```GET```</strong> request gathers a resource from a server. ```GET``` variables are easy to determine from a PHP website, as they appear in the URL of a website. A ```GET``` variable can be obtained by observing a ```?``` after a ```.php``` extension in the URL, then each subsequent name with a ```=``` is the name and value of each ```GET``` variable.

Here is an example: ```https://www.testwebsite.com/index.php?name=umdclassXXXX&age=21```, where ```name``` is umdclassXXXX and ```age``` is 21.

If you have ```telnet``` installed, you can call a ```GET``` request for the landing page of any website. Simply type ```telnet url 443``` to call an HTTPS request for any provided URL, then type ```GET /``` to get the HTML source code for the landing page of the website.

#### <center><strong>Differences Between Both Protocols</strong></center>

| Feature        | GET Method                                                                 | POST Method                                                                            |
|----------------|----------------------------------------------------------------------------|----------------------------------------------------------------------------------------|
| Operation      | Used to retrieve information from the server.                              | Used to send data to the server to create/update a resource.                           |
| Data Location  | Appends data to the URL, visible to all.                                   | Includes data in the request body, not displayed in the URL.                           |
| Idempotency    | Idempotent; the same request can be repeated with no further changes.      | Non-idempotent; repeating the same request can lead to different results.              |
| Data Size      | Limited by the URL length; less data can be sent.                          | No limitations on data size; suitable for large amounts of data.                       |
| Caching        | Can be cached.                                                             | Not cached by default.                                                                 |
| Security       | Less secure as data is exposed in the URL.                                 | More secure; data is concealed within the request body.                                |
| Use Case       | Ideal for searching and retrieving data.                                   | Ideal for transactions and updating data.                                              |


<center><em>Source: <a href="https://www.akto.io/academy/get-vs-post">Atko.io</a></center>

<br>

In PHP, making a ```POST``` request will refresh the page, creating ```$_POST["name"] = key``` variables. These variables go away when the page is refreshed, so you may call them within an "if" statement where ```$_SERVER['REQUEST_METHOD'] == 'POST'```. When PHP detects a ```POST``` request, the PHP file will enter this code segment, where you may process the ```POST``` variables. 

Similarly with ```GET```, you may call ```$var = $_GET["key"]```, allowing PHP to retrieve variables from the address bar.

### Step 1: Identifying Your Authentication Token

<strong>This lab is going to require port forwarding, which requires setup on your own, local machine. If you have not already done port forwarding, you will need to follow the steps from this notebook.</strong>

<strong>Additionally, this lab will contain two nodes.</strong> Attempting to access a node called ```xss``` will not work! The two nodes for this lab are ```server``` and ```client```. <u>You will be doing all of your work on the ```server``` node.</u>

After you have port forwarded to your ```server``` node, access ```http://localhost:port/xss_practice.php```, where ```port``` is the number that you used in your SSH statement.

<strong>For example</strong>, if you have already set up port forwarding, type the following command into your terminal: ```ssh -L 8080:server:80 USERNAME_GOES_HERE-xdc-USERNAME_GOES_HERE```, then access the practice page by typing ```http://localhost:8080/xss_practice.php``` into your web browser.

Once you have navigated to this website, you are going to land on a notes page. The notes page is where users may freely post whatever note that they want, as well as stating who has posted the note.

You already have an account for this website. These are your credentials:

| Username: | Password:    |
|-----------|--------------|
|  umdsec   |   hacker123  |

Using these credentials, sign into the website.

<strong>This website's design is very insecure, and you will learn how to break it.</strong> Currently, the website does authentication through the address bar. Every user that has an account has a random authentication token that's 32 characters long. Currently, cookies and sessions will not be introduced yet for authentication. Once you learn the basics of cross-site scripting, you will practice cross-site scripting with cookies and sessions.

You can find the authentication token inside of the URL of your website. <u>This is what allows you to access your profile.</u> Changing this token will sign you out, unless you provide someone else's token, which is the goal for this topic.

Sign into the website using the credentials that are provided above, then locate the authentication token inside of the address bar. Copy and paste the value of this button into the field below.

In [4]:
# Click the button below to check your work.
step1Complete = False

# Function to check if the student's answer was correct.
def step_1():
    # Important variables that must be accessed outside of this function.
    global step1Complete, result

    # Loading, in case the check is slow.
    with output1:
        output1.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput1.value == ""):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step1Complete = False

    else:
        # Construct the SSH command for testing the SQL query.
        auth_token = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_1.py 1 {userInput1.value}" >/dev/null """
        result = subprocess.run(auth_token, shell=True)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo {userInput1.value} > /home/.checker/responses/step_1_answer.txt" >/dev/null """
        save_result = subprocess.run(save_command, shell=True)
        
        if (result.returncode == 1):
            output1.clear_output()
            with output1:
                display(HTML("<span style='color: green;'>You correctly identified your authentication token!</span>"))
                step1Complete = True
        
        elif (result.returncode == 0):
            output1.clear_output()
            with output1:
                display(HTML("<span style='color: red;'>The authentication token you provided for umdsec does not match. Check again.</span>"))
                step1Complete = False
        
        elif (result.returncode == 2):
            output1.clear_output()
            with output1:
                display(HTML("<span style='color: red;'>An error occurred when checking this step. Please contact your professor or TA.</span>"))
                step1Complete = False

def check_step_1(b):
    if (warn_student()):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_1()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("1", result.returncode, userInput1.value)
    
# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading1 = widgets.Output()
display(loading1)
with loading1:
    loading1.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput1 = widgets.Text(
    placeholder='Type your auth token here',
    description='Token:',
    layout=widgets.Layout(width='75%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_1_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput1.value = result.stdout

# After the student's response was loaded, clear the output.
loading1.clear_output()

# Creating the button.
button = widgets.Button(description="Check Auth Token")

# Creating an output area.
output1 = widgets.Output()

# Run the command on click.
button.on_click(check_step_1)

# Display the output.
display(userInput1, button, output1)

Output()

Text(value='', description='Token:', layout=Layout(width='75%'), placeholder='Type your auth token here')

Button(description='Check Auth Token', style=ButtonStyle())

Output()

## <strong>Topic 2: Session Hijacking</strong>

In this topic, you are going to practice HTTP protocols from Topic 1, and practice how to steal someone's authentication token. This token will allow you to access someone else's account. With the poor design of the website, you are able to access someone else's account by changing the token found in the URL of the website.

XSS attacks require some inner knowledge with how websites and its server operate. Since you are working in a learning environment, you will be told how ```xss_practice.php``` is setup.

```xss_practice.php``` uses two SQL databases:
- ```users``` stores the username and password of each user. Additionally, a random token is assigned to the user which is used for authentication.
- ```notes``` stores the notes and the username of the person who wrote the note.

When a note is saved, the memo is written to the database, and can only be viewed by that user. Currently, you have written one note that is already written to the website, and you may delete notes, if you wish. <u>It's not necessary to understand how to use SQL for this lab.</u> It's only important to understand that each note is associated with the username in the database.

With poor website design, signing into your account will display your authentication token within the URL of the website. The creator of this website thinks that you do not know what this string is. If this authentication token is changed, you will no longer be identified as ```umdsec```, OR you may be identified as someone else.

Your goal for this topic is figuring out how to write a victim's authentication token to a note on your own account. Then, signing into their account and viewing private information that they've stored.

### Step 2: Using Website to Execute JavaScript

In HTML, forms are user-friendly POST requests. You're going to learn how to handle POST requests by injecting JavaScript onto the website, instead of filling out the form.

You are free to open ```xss_practice.php``` in your ```server``` node and observe how the form works for uploading a note to the website. This is optional, in case you'd like to understand how HTTP requests work in HTML. Upon signing into the website, there is an HTML element called ```<form>```, which is named ```noteForm```. ```<form>``` elements have ```<input>``` fields that have different types to them. There are two input tags that are within this form.

- There is a ```<textarea>``` that is named ```note```, where you may type in a note that gets sent to the database.
  - The format for this in your POST request is: ```note=YourNoteHere```, where ```YourNoteHere``` is the value of the ```<textarea>```.
- There is a ```<input>``` field with the type ```hidden```. The ```hidden``` field is named ```username```.
  - The format for this in your POST request is: ```username=umdsec```, where ```umdsec``` is the value of the ```<input type="hidden">```.
  - These ```hidden``` types are form fields that are not viewable within the form. They contain information that are pre-determined by the website, which in this case, is the username.

<u>Although the input is type ```hidden```, you can still see this field in the source code of the HTML. You may find it yourself by using Inspect Element.</u> 

<figure><center><img src="resources/xss/inspect_element.png" style="width: 75%; height: 75%;"></img></center></figure>

To access Inspect Element on your browser, right-click on the ```xss_practice.php``` website, and look for a button called "Inspect" or "Inspect Element". Some browsers may have Inspect Element under a "Developer Tools" button. If you're stuck, you will need to use a search engine to find instructions for your specific browser.

When a form is submitted, a ```POST``` request is sent to ```xss_practice.php```, where the server takes the ```POST``` information from the form, then writes it to the database by safely using prepared statements. SQL injection will not work in this lab, so you will need to use cross-site scripting.

First, you are going to find the security vulnerability for this website.

<strong>Your task:</strong> Write a ```<script>``` block as a note on the website. Within the script block, use ```console.log()``` to write the value of ```2 * 31```.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>

- Use Inspect Element on your browser to view the "Console". The console is where you will be able to view the result of your payload.
- Since your payload is containing a ```<script>``` tag, it will not render. If you are not seeing a message displaying on the website, then your payload is working.

<span style="color: orange"><strong><img src="resources/idea.png" style="width: 12px"> What is "Use Previous Work"?</strong></span> In the ```xss_practice.php``` website, you are given the option to reset the notes on your account or the victim's account. However, at each step, you may wish to delete payloads, so that you are not running previous exploits as you work within the notebook.

By checking "Use Previous Work", your notebook will check to see if you have already completed this step. If you have previously completed the step, it will check the last response that your notebook has saved. <u>This checkbox will auto-check itself when you complete a step successfully, so that your answer is locked in.</u> If you run a step without checking this box, it will read any existing payload(s) that are currently stored on ```xss_practice.php```.

In [5]:
# Click the button below to check your work.
step2Complete = False

# Function to check the permissions.
def step_2():
    # Important variables that must be accessed outside of this function.
    global step2Complete, result
    
    with output2:
        output2.clear_output()
        display(HTML("<span>This step will take some extra time to check. Please wait! <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # Before checking the step, see what the value of "Use Previous Work" is.
    check_prev_work = prev_work_2.value

    # This will be appended to the result subprocess. If check_prev_work is True, then just add one.
    prev_work = 0
    
    if (check_prev_work or runAllSteps):
        prev_work += 1

    # Run the script.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_2.py 2 ' + str(prev_work) + '"', shell=True, capture_output=True, text=True)

    if (result.returncode == 1):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True

            # Mark "Use Previous Work" as true to avoid progress loss.
            prev_work_2.value = True

    elif (result.returncode == 0):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>The result of 2*31 cannot be found in console.log upon accessing the site. Check your payload.</span>"))
            step2Complete = False
    
    elif (result.returncode == 2):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>An error occurred when checking this step. Please contact your professor or TA.</span>"))
            step2Complete = False

def check_step_2(b):
    if (warn_student()):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_2()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("2", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating a checkbox for "Use Previous Work".
prev_work_2 = widgets.Checkbox(value=False,
    description='Use Previous Work',
    disabled=False,
    indent=False)

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(button, prev_work_2, output2)

Button(description='Check Payload', style=ButtonStyle())

Checkbox(value=False, description='Use Previous Work', indent=False)

Output()

### Step 3: Discovering an Exploit

Recently, you discovered another user who exists on the website who managed to take the username: ```Hacker```. You want to attempt to access their account without knowing the password for it. Currently, this is what you know about the website:

- Uploading a note to the website only requires the username and note content through a POST request.
- You cannot view notes that are made by other users.
- JavaScript is freely executing through your notes that you upload.

<strong>Using this information, an opportunity arises.</strong> You can post notes onto someone else's account, just as long as you know the username of the account. You can execute a POST request in JavaScript, which contains the username and a note, and it will write it to that user's account.

<figure><center><img src="resources/xss/topic_2_example_1.png" style="width: 75%; height: 75%;"></img></center></figure>

JavaScript has built-in functions for calling HTTP requests by using either ```fetch``` or ```XMLHttpRequest```. The ```fetch``` statement will be used as an example, since ```XMLHttpRequest``` requires multiple functions to be called. Here is a template of a ```fetch``` statement that can be used to ```POST``` information to ```xss_practice.php```:

```
<script>
  fetch("xss_practice.php", {
    method: "POST",
    headers: {
      "Content-Type": "application/x-www-form-urlencoded"
    },
    body: "key=value"
  });
</script>
```

A breakdown:
- ```method``` allows you to choose your HTTP request. In this case, a ```POST``` request.
- ```headers``` allow you to create additional instructions for your HTTP request.
- ```Content-Type``` indicates what type of information is being sent within the body of the request. In this case, ```application/x-www-form-urlencoded``` encodes key-value pairs through HTML requests. You can read more about these from <a href="https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Content-Type">Mozilla's documentation</a>.
- ```body``` stores the information that you're sending with the request.

<strong>This method works best when you are passing one word or value to a request</strong>. It can become difficult to send a ```fetch``` request if you have more than one key, or a larger value that you wish to send. If you're familiar with JSON, you may use JSON in your payload. However, this notebook does not expect you to know JSON. <u>We will just be using ```urlencoded``` for this lab.</u>

<strong>Your task:</strong> Post a note with a ```<script>``` tag that executes the following information within your account:
- Create a ```POST``` request to the ```xss_practice.php``` page which contains the following information:
  - The ```username``` with the value ```Hacker```.
  - A ```note``` with the value of anything you'd like to send to ```Hacker```. Such as: "I'm in your account." or "This website is unsafe!" You may use whatever note you'd like.
    - Do not use ```alert()``` or ```console.log()``` for your message to ```Hacker```. Just include content within the ```note``` parameter.

This is what your attack should be doing:
- Your payload gets posted as a note to your website, which should not display because your ```<script>``` tag is being rendered in HTML.
- Each time that your website gets refreshed, your ```<script>``` tag is executing, which creates a note on ```Hacker```'s, account.
- ```Hacker``` will see a new note posted with your content posted on it.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>
- You need to sent two key-value pairs in the body of the HTTP request. You may separate these with ```&``` in between the two key-value pairs within the body of your call.
  - If you wish to add spaces in your note, you may use ```+```, since spaces are not interpreted in ```urlencoded```.
- You are <strong>strongly encouraged</strong> to try copying the contents from the template above and try executing this code directly within the Console of your browser. Once you have a command working, copy/paste it into a note surrounded by ```<script>``` tags, then post the note. The check for this step will make sure that you have this note posted on the website.
  - Your browser may warn you about copy/pasting code into your console, due to security. If this occurs, your browser will give you a command to run to allow it, such as ```allow pasting```. Read your browser's instructions to allow this.
- If you are running into troubles with your code, <u>you may view the Console</u> within your browser to view JavaScript errors.

In [6]:
# Click the button below to check your work.
step3Complete = False

# Function to check the permissions.
def step_3():
    # Important variables that must be accessed outside of this function.
    global step3Complete, result

    with output3:
        output3.clear_output()
        display(HTML("<span>This step will take some extra time to check. Please wait! <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Before checking the step, see what the value of "Use Previous Work" is.
    check_prev_work = prev_work_3.value

    # This will be appended to the result subprocess. If check_prev_work is True, then just add one.
    prev_work = 0
    
    if (check_prev_work or runAllSteps):
        prev_work += 1

    # Run the script.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_2.py 3 ' + str(prev_work) + '"', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step3Complete = True

            # Mark "Use Previous Work" as true to avoid progress loss.
            prev_work_3.value = True

    elif (result.returncode == 0):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Accessing umdsec's account did not create a new note for the victim. Try again.</span>"))
            step3Complete = False
    
    elif (result.returncode == 2):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>An error occurred when checking this step. Please contact your professor or TA.</span>"))
            step3Complete = False

def check_step_3(b):
    if (warn_student()):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_3()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("3", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating a checkbox for "Use Previous Work".
prev_work_3 = widgets.Checkbox(value=False,
    description='Use Previous Work',
    disabled=False,
    indent=False)

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(button, prev_work_3, output3)

Button(description='Check Payload', style=ButtonStyle())

Checkbox(value=False, description='Use Previous Work', indent=False)

Output()

### Step 4: Stealing Someone's Information

Now, you are successfully able to post content on someone else's account by injecting JavaScript as a note on the website. At the moment, your attack is not stealing data from users.

From the previous step, you have a working payload that allows you to post whatever you want onto someone else's account. Now, whenever ```Hacker``` accesses their website, you want to deliver a ```POST``` request that sends their URL value to your OWN account. Their URL contains their authentication token, allowing you to sign-in as ```Hacker```.

With some careful thought, this is the outline that you plan to use:
- Write a ```fetch``` request that writes a note onto ```Hacker```'s account. <u>You know how to do this already from Step 3.</u>
- ```Hacker``` opens the page and executes infected JavaScript code that you wrote to their account.
- The infected JavaScript code retrieves the current URL of ```Hacker```'s account, which contains their authentication token.
- Your payload takes ```Hacker```'s URL, then ```POST```s it as a note to your account.

<figure><center><img src="resources/xss/topic_2_example_2.png" style="width: 75%; height: 75%;"></img></center></figure>

<strong>Your task</strong>: Create a payload that will send a note containing infected JavaScript to ```Hacker```'s account. The infected JavaScript will take the current URL of their profile, then send it back as a note to your own profile.

This step is going to require some creative thinking. A template will be provided to you, then broken down to help demonstrate how it works.

```
<script>
  fetch("xss_practice.php", {
    method: "POST",
    headers: {
      "Content-Type": "application/x-www-form-urlencoded"
    },
    body: 'username=EnterUserHere&note=' + encodeURIComponent(`<script>
             Insert your payload here.
             It must be a fetch() statement!
           <\/script>`)
  });
</script>
```

This ```fetch``` function works mostly the same as before. However, there are some important instructions to understand with this template:
- Special characters inside of a POST request, such as ```, / ? : @ & = + $ #``` can break the data that's being sent. When you wrap your payload with ```encodeURIComponent```, these characters will be safely escaped.
- You will need to wrap quotes around your ```encodeURIComponent``` function, since it takes a string for an argument. Instead of quotes, you may also backticks (``` ` ```), like the template uses. That way, you do not need to worry about escaping single/double quotes.
- When nesting ```<script>``` tags within each other, the first occurrence of a ```</script>``` will close off the outer-most ```<script>``` tag. To prevent this, use ```<\/script>``` when closing off the inner ```<script>``` tag.

Some additional tips are given below for when you construct your payload.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>

- You are <strong>strongly recommended</strong> to open the Console of your browser and attempt to type a ```fetch()``` request that will post a note to the victim's account. Use the ```Peek``` button to view any payloads/notes that ```Hacker``` will execute/read when they sign in. Once you have a working payload, you can use this for the body of a note within another ```fetch``` request.
- ```location.href``` returns the URL of the current page where the JavaScript is executing. Use this to get the URL of ```Hacker```'s account.
- Including a URL within a ```POST``` request can cause undefined behavior, since they likely contain unsafe characters that can break your HTTP request. When handling the URL of ```Hacker```'s website, use ```encodeURIComponent``` on the URL to safely escape those characters within the body of your victim's ```POST``` statement.

In [7]:
# Click the button below to check your work.
step4Complete = False

# Function to check the permissions.
def step_4():
    # Important variables that must be accessed outside of this function.
    global step4Complete, result

    with output4:
        output4.clear_output()
        display(HTML("<span>This step will take some extra time to check. Please wait! <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Before checking the step, see what the value of "Use Previous Work" is.
    check_prev_work = prev_work_4.value

    # This will be appended to the result subprocess. If check_prev_work is True, then just add one.
    prev_work = 0
    
    if (check_prev_work or runAllSteps):
        prev_work += 1

    # Run the script.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_2.py 4 ' + str(prev_work) + '"', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: green;'>Success! Your payload made hacker's authentication token leak onto your notes page. Try accessing it for the next step.</span>"))
            step4Complete = True

            # Mark "Use Previous Work" as true to avoid progress loss.
            prev_work_4.value = True

    elif (result.returncode == 0):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>The victim's account was accessed, but did not print their URL on your page. Try again.</span>"))
            step4Complete = False
    
    elif (result.returncode == 2):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>An error occurred when checking this step. Please contact your professor or TA.</span>"))
            step4Complete = False

def check_step_4(b):
    if (warn_student()):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_4()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("4", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating a checkbox for "Use Previous Work".
prev_work_4 = widgets.Checkbox(value=False,
    description='Use Previous Work',
    disabled=False,
    indent=False)

# Creating an output area.
output4 = widgets.Output()

# Run the command on click.
button.on_click(check_step_4)

# Display the output.
display(button, prev_work_4, output4)

Button(description='Check Payload', style=ButtonStyle())

Checkbox(value=False, description='Use Previous Work', indent=False)

Output()

### Step 5: Accessing Someone's Information

In the previous step, you successfully stole the URL of ```Hacker```'s account. Now, you may access their account and read their notes.

Copy and paste the authentication token of the victim's URL, then replace your authentication token with this. <strong>Note</strong>: The full URL will not work because ```10.0.1.1``` is the ```localhost``` of the ```server``` node. You will need to keep ```localhost:port``` in the address bar. 

Using the blank below, type in ```Hacker```'s credit card number that they stored on the website.

In [8]:
# Click the button below to check your work.
step5Complete = False

# Function to check if the student's answer was correct.
def step_5():
    # Important variables that must be accessed outside of this function.
    global step5Complete, result

    # Loading, in case the check is slow.
    with output5:
        output5.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput5.value == ""):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step5Complete = False

    else:
        # Construct the SSH command for testing the SQL query.
        auth_token = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_2.py 5 {userInput5.value}" >/dev/null"""
        result = subprocess.run(auth_token, shell=True)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo {userInput5.value} > /home/.checker/responses/step_5_answer.txt" >/dev/null"""
        save_result = subprocess.run(save_command, shell=True)
        
        if (result.returncode == 1):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
                step5Complete = True
        
        elif (result.returncode == 0):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: red;'>The card number you provided does not match the number in the victim's account. Check again.</span>"))
                step5Complete = False
        
        elif (result.returncode == 2):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: red;'>An error occurred when checking this step. Please contact your professor or TA.</span>"))
                step5Complete = False

def check_step_5(b):
    if (warn_student()):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_5()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("5", result.returncode, userInput5.value)
    
# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading5 = widgets.Output()
display(loading5)
with loading5:
    loading5.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput5 = widgets.Text(
    placeholder='Type the victim\'s credit card number here',
    description='Number:',
    layout=widgets.Layout(width='75%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_5_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput5.value = result.stdout

# After the student's response was loaded, clear the output.
loading5.clear_output()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output5 = widgets.Output()

# Run the command on click.
button.on_click(check_step_5)

# Display the output.
display(userInput5, button, output5)

Output()

Text(value='4816284615375930\n', description='Number:', layout=Layout(width='75%'), placeholder="Type the vict…

Button(description='Check Work', style=ButtonStyle())

Output()

## <strong>Topic 3: PHP Sessions and ```document.cookie```</strong>

#### <strong>PHP Sessions</strong>
PHP is a server-side language used in web development. PHP handles back-end development tasks such as executing SQL queries for databases, managing transactions, saving data to storage containers, and much more. For this lab, we are focusing on some important concepts for handling user information: sessions and cookies.

<strong>Sessions</strong> are stored on a server and expire when the browser is closed. Sessions are generally safer and store data for a shorter duration. However, they do not make you invulnerable, as you will discover later in the lab.

<strong>Cookies</strong> are stored on the client's browser and have a set expiration time. Cookies are stored as text files, which the browser retrieves when a website requests them. You can view where your browser stores cookies and examine their contents. A hacker can retrieve cookies in the same way that you can.

<em>Side note:</em> The European Union requires websites to be GDPR compliant if they use cookies, which is one of the strictest privacy and security laws in the world. If you are interested in reading more about cookies and the GDPR regulation, you can read more <a href="https://gdpr.eu/cookies/">here</a>. 

It should be clear that cookies are not intended for storing private data. A safer alternative to cookies is sessions. However, cookies can still be used for storing data that is meant to be public.

Sessions and cookies are superglobal variables in PHP, meaning they are accessible across all PHP files on a server. However, a PHP file must "activate" a session by using ```session_start()```. They are accessed using ```$_SESSION["key"] = value;``` or ```$_COOKIE["key"] = value;```. Before using sessions in PHP, you must call ```session_start()```. Sessions can be manually ended by using ```session_destroy()```.

#### <strong>```document.cookie```</strong>
The ```document.cookie``` property is a <strong>JavaScript</strong> property. JavaScript is a language used in browsers, which means you can debug websites that use JavaScript directly from your browser. Each browser is different, but generally, you can access the console by following these steps:

- Right-click on a webpage (this does not work on all webpages, such as JupyterLab).
- Navigate to "Inspect Element" or "Inspect".
- Select the "Console" tab.

Since ```document.cookie``` is a property, you can simply type ```document.cookie``` into the console to view its value.

Whenever PHP sets a session variable, the user is assigned a value called ```PHPSESSID```. Although session variables are stored on the server, the session ID must be stored in the browser, typically as a cookie named ```PHPSESSID```. When a request is made to the server, it uses this session ID to identify the corresponding session, retrieve the associated variables, and make them available to that user. These session variables cannot be accessed by anyone else unless they have your session ID.

### Step 6: Finding ```document.cookie```

When you access ```xss_practice.php``` in your browser, try typing ```document.cookie``` within the Console of your browser. Why is ```PHPSESSID``` missing? 

Remember to read about PHP sessions before answering this question.

In [9]:
# Type your response below.
step6Complete = False

# Function to save the short answer.
def step_6():
    # Important variables that must be accessed outside of this function.
    global step6Complete, result

    # Loading, in case saving is slow.
    with output6:
        output6.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    if (userInput6.value == ""):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>You did not type a response.</span>"))
            step6Complete = False

    else:
        # Replace backticks with single quotes in the input.
        safe_value = userInput6.value.replace("`", "'")
        
        # Build the SSH command to write to the file using 'cat' reading from stdin.
        ssh_command = [
            "ssh",
            "-i", "/home/USERNAME_GOES_HERE/.ssh/merge_key",
            "USERNAME_GOES_HERE@server",
            "cat > /home/.checker/responses/step_6_answer.txt"
        ]
        
        # Pass safe_value as input so that no shell quoting is needed.
        result = subprocess.run(ssh_command, input=safe_value, text=True)
        
        if result.returncode == 1:
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>There was an error saving your response.</span>"))
                step6Complete = False
                
        elif result.returncode == 0:
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: green;'>Your response was saved.</span>"))
                step6Complete = True

def check_step_6(b):
    if warn_student():
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_6()
        if not runAllSteps:
            safe_value = userInput6.value.replace("`", "'")
            user_input_quoted = shlex.quote(safe_value)
            trigger_save("6", 1 if result.returncode == 0 else 0, user_input_quoted)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading6 = widgets.Output()
display(loading6)
with loading6:
    loading6.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area. We will need to assign output of process to the value of this input.
userInput6 = widgets.Textarea(
    placeholder='Type your response here',
    description='Response:',
    layout=widgets.Layout(width='75%', height='150px', margin='10px')
)

def on_input_change(change):
    # Replace backticks with single quotes in the new value.
    new_val = change['new'].replace("`", "'")
    # Only update if there's a change (to avoid unnecessary recursion).
    if new_val != change['new']:
        userInput6.value = new_val

userInput6.observe(on_input_change, names='value')

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_6_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Output creates a newline. Remove it.
userInput6.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading6.clear_output()

# Creating the feedback output area.
output6 = widgets.Output()

# Creating the button.
button = widgets.Button(description="Save Response")

# Run the command on click.
button.on_click(check_step_6)

# Display the output.
display(userInput6, button, output6)

Output()

Textarea(value='Hello!', description='Response:', layout=Layout(height='150px', margin='10px', width='75%'), p…

Button(description='Save Response', style=ButtonStyle())

Output()

## <strong>Topic 4: A Large-Scale Application of Cross-Site Scripting</strong>

The developers of the notes website has now moved to another project called Sloth's Unlimited. After learning about the security vulnerability on their previous website, they decided to design a new website with more added security. In hopes of avoiding another security exploit, they asked you to try breaking the website with your knowledge in cross-site scripting.

<strong>You are strongly encouraged to read about the outline of the website before continuing to the first step in this topic.</strong>

There are three nodes in this lab: ```client```, ```server```, and ```teshwan```.

- The ```server``` node hosts the website. This is the node where all of the PHP files are stored, and this is the node that you will be using for the rest of the lab.
- The ```client``` node is where your notebook is accessing the website from. The ```client``` is the root user of the website, which holds the cookie that you wish to steal.
- The ```teshwan``` node is Lord Teshwan's computer. Teshwan is a developer for Sloth's Unlimited. Teshwan's computer is infected with a file called ```steal.php```, which he doesn't know about. However, you know about this infected file. This file accepts a cookie for a GET parameter. Recall from Topic 1 that GET parameters are variables that are accessed from the URL of the website.

These are the following IP addresses that will be important to know for the lab:

| Nodes:        | ```client```   | ```server```   | ```teshwan```  |
|---------------|----------------|----------------|----------------|
| IP Addresses: | ```10.0.1.1``` | ```10.0.1.2``` | ```10.0.1.3``` |

On the website, you will need to create an account to sign into the forum. Once an account is made, you may begin posting. You may choose whatever topic that you would like to post on, or create your own topic.

```client```, ```server```, and ```teshwan``` are on the same network, which means that you can call requests to access each other's web files by using cURL requests (not required for the lab). This is the topology of the XSS lab:

<figure><center><img src="resources/xss/topology.png" style="width: 25%; height: 25%;"></img></center></figure>

<u>Here is the ultimate goal of the following few steps:</u> You are going to write a payload script that ```client``` will access whenever you click "Check Payload". Since ```steal.php``` is a web file that's stored in ```/var/www/html``` on ```teshwan```'s node, you will need to write a payload that will redirect ```client``` to ```http://10.0.1.3/steal.php?cookie=COOKIE_VALUE```. This is Teshwan's IP address, which is accessible by ```client```.

<u>Here are some important considerations when writing your payload</u>:
- You only need to access the ```server``` node for the remainder of the lab, like you have been doing already.
  - If you haven't already, you will need to port forward from the ```server``` node. Do not port forward from other nodes or a ```xss``` node. An ```xss``` node doesn't exist.
- Using an ```POST``` request will not work. It must be ```GET```. More on this is below.
- The ```COOKIE_VALUE``` is the entirety of ```document.cookie``` in JavaScript. You are not required to split the variable or use regular expressions.

<strong>Why will a POST request not work?</strong> Secure websites use CORS, which stand for "Cross-Origin Resource Sharing". CORS is a security feature built into a browser that will prevent images, videos, files, and more, from being shared across different websites. CORS prevents random websites from using your API to intiate POST requests without being on your website. This means making POST requests to other domains (or IP addresses, in this case) are blocked. The reason why a POST request worked in the first section is because all POST requests were made within the same domain of the website. Since there are three different IP addresses, with an infected payload running on a different IP from the ```server```, CORS will prevent ```client``` from calling something on ```teshwan```'s IP address. Instead, we must redirect ```client``` to ```teshwan```'s IP address and force ```client``` to call a malicious script.

### Step 7: Testing Your Exploit

Navigate to ```localhost:port/index.php``` to access Sloth's Unlimited. Once again, <strong>port forwarding is required</strong> in order to access the website from your browser. ```port``` is a value that you chose in your SSH statement.

Currently, there is not a sanitization function applied to the website, making the entire forum prone to cross-site scripting. When a message gets created, this is what the message gets called through:

```
function sanitize($string) {
  return $string;
}
```

This function is stored in ```/var/www/html/sanitize.php```. You do not need to open this file quite yet.

Type a payload into the website that causes ```client``` to call ```teshwan```'s ```steal.php``` file. After you have a working payload, you should see the ```Eagles``` category appear on the home page of Sloth's Unlimited. 

<span style="color: orange"><strong><img src="resources/idea.png" style="width: 12px"> What is "Reset Entire Forum"?</strong></span>

This button will reset the forum back to its original state so that you may test a new payload. When you're posting infected payloads, these payloads will stay unless you delete them. This button will be useful for testing different payloads, and should be used between each step so that you can test your exploits easier.

<span style="color: orange"><strong><img src="resources/idea.png" style="width: 12px"> What is "Use Previous Work"?</strong></span>

Like from Topic 1, resetting the forum will erase your payloads. Whenever your work is tested, its response will be automatically saved so that you can pass this step later. When checking "Use Previous Work", your previous response to the step will be used and see if you previously passed it.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>
- From the first topic, you learned how to use ```fetch``` to send a POST request. However, this approach is simply finding a way to make ```client``` call ```http://10.0.1.3/steal.php?cookie=COOKIE_VALUE```, where ```COOKIE_VALUE``` is the value of ```document.cookie```. The ```10.0.1.3``` IP address is Lord Teshwan's computer, which the ```client``` node has access to.
  - You may either find a way to call a GET request with ```fetch```. Or, even easier, use JavaScript to redirect the ```client``` onto another page.
  - A redirect in JavaScript is done by setting ```window.location.href``` equal to the URL that you're redirecting the user to.
- Send the entire ```PHPSESSID=XXXX``` value to ```COOKIE_VALUE```. Do not try to send only the value of PHPSESSID itself. No need to use string manipulation for your payload.
- Like before, a ```<script>``` block will not appear on your website. However, when YOU access the infected page, it will not execute your payload. ```10.0.1.3``` is not an accessible IP address on your computer, so your web browser is not redirecting you to this domain. It's only accessible through ```client``` (this notebook) and the ```server```. This is ```teshwan```'s IP address.

In [10]:
# Click the button below to check your work.
step7Complete = False

# Function to check the permissions.
def step_7():
    # Important variables that must be accessed outside of this function.
    global step7Complete, result

    with output7:
        output7.clear_output()
        if (prev_work_7.value == 0):
            display(HTML("<span>Accessing all topics on your forum. Please wait. This will take more time than the previous topics... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

        elif (prev_work_7.value == 1):
            display(HTML("<span>Checking your previous response. Please wait... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Before checking the step, see what the value of "Use Previous Work" is.
    check_prev_work = prev_work_7.value

    # This will be appended to the result subprocess. If check_prev_work is True, then just add one.
    prev_work = 0
    
    if (check_prev_work or runAllSteps):
        prev_work += 1

    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_4.py 7 ' + str(prev_work) + '"', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: green;'>Success! The Eagles page was automatically created. Stealing the admin's cookie was successful.</span>"))
            step7Complete = True

            # Automatically check the checkbox so that students don't accidentally run this step later and lose their progress.
            prev_work_7.value = True

    elif (result.returncode == 0):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>Every page on the forum was accessed, but Eagles wasn't created on the home page. This means that the admin's cookie was not stolen successfully. Try again.</span>"))
            step7Complete = False
    
    elif (result.returncode == 2):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>An error occurred when checking this step. Please contact your professor or TA.</span>"))
            step7Complete = False

    elif (result.returncode == 3):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>It appears this step wasn't attempted. Please run this step without \"Use Previous Work\" checked.</span>"))
            step7Complete = False

def check_step_7(b):
    if (warn_student()):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_7()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("7", result.returncode)

def reset_forum(b):
    subprocess.run("ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server /home/.checker/reset.sh", shell=True)
    with output7:
        output7.clear_output()
        display(HTML("<span style='color: black;'>Your forum was reset. All new messages have been deleted.</span>"))

    # Auto-save.
    if (not runAllSteps):
        trigger_save("7", result.returncode)

# Creating the buttons.
button = widgets.Button(description="Check Payload")
reset_button = widgets.Button(description="Reset Entire Forum")

# Creating a checkbox for "Use Previous Work".
prev_work_7 = widgets.Checkbox(value=False,
    description='Use Previous Work',
    disabled=False,
    indent=False)

# Creating an output area.
output7 = widgets.Output()

# Run the commands on click.
button.on_click(check_step_7)
reset_button.on_click(reset_forum)

# Display the output.
display(button, reset_button, prev_work_7, output7)

Button(description='Check Payload', style=ButtonStyle())

Button(description='Reset Entire Forum', style=ButtonStyle())

Checkbox(value=False, description='Use Previous Work', indent=False)

Output()

### Step 8: Breaking The First Sanitization Function

The developers of Sloth's Unlimited have decided to use a sanitization function for all posts that are made to their message board. This is the sanitization function that they have decided to use:

```
function sanitize($string) {
  return preg_replace("/<\/?script[^>]*>/", "", $string);
}
```

This function will take a single occurrence of ```<script>``` and ```</script>```, then replace it with an empty string.

Adjust your payload so that it will break this sanitization function. Click "Check Payload" once you have submitted a payload and would like to test it.

<span style="color: orange"><strong><img src="resources/idea.png" style="width: 12px"> Note:</strong></span> After completing the previous step, the Eagles category was created and your previous payload is already on the website. To start with a fresh forum, click "Reset Entire Forum" before typing in a new payload. The Eagles category will be wiped upon testing, but your previous payload is patched in this step. It would be best to reset the website as you complete each step.

In [11]:
# Click the button below to check your work.
step8Complete = False

# Function to check the permissions.
def step_8():
    # Important variables that must be accessed outside of this function.
    global step8Complete, result

    with output8:
        output8.clear_output()
        if (prev_work_8.value == 0):
            display(HTML("<span>Accessing all topics on your forum. Please wait. This will take more time than the previous topics... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

        elif (prev_work_8.value == 1):
            display(HTML("<span>Checking your previous response. Please wait... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Before checking the step, see what the value of "Use Previous Work" is.
    check_prev_work = prev_work_8.value

    # This will be appended to the result subprocess. If check_prev_work is True, then just add one.
    prev_work = 0
    
    if (check_prev_work or runAllSteps):
        prev_work += 1

    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_4.py 8 ' + str(prev_work) + '"', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: green;'>Success! The Eagles page was automatically created. Stealing the admin's cookie was successful.</span>"))
            step8Complete = True

            # Automatically check the checkbox so that students don't accidentally run this step later and lose their progress.
            prev_work_8.value = True

    elif (result.returncode == 0):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>Every page on the forum was accessed, but Eagles wasn't created on the home page. This means that the admin's cookie was not stolen successfully. Try again.</span>"))
            step8Complete = False
    
    elif (result.returncode == 2):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>An error occurred when checking this step. Please contact your professor or TA.</span>"))
            step8Complete = False

    elif (result.returncode == 3):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>It appears this step wasn't attempted. Please run this step without \"Use Previous Work\" checked.</span>"))
            step8Complete = False

def check_step_8(b):
    if (warn_student()):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_8()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("8", result.returncode)

def reset_forum(b):
    subprocess.run("ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server /home/.checker/reset.sh", shell=True)
    with output8:
        output8.clear_output()
        display(HTML("<span style='color: black;'>Your forum was reset. All new messages have been deleted.</span>"))

    # Auto-save.
    if (not runAllSteps):
        trigger_save("8", result.returncode)

# Creating the buttons.
button = widgets.Button(description="Check Payload")
reset_button = widgets.Button(description="Reset Entire Forum")

# Creating a checkbox for "Use Previous Work".
prev_work_8 = widgets.Checkbox(value=False,
    description='Use Previous Work',
    disabled=False,
    indent=False)

# Creating an output area.
output8 = widgets.Output()

# Run the commands on click.
button.on_click(check_step_8)
reset_button.on_click(reset_forum)

# Display the output.
display(button, reset_button, prev_work_8, output8)

Button(description='Check Payload', style=ButtonStyle())

Button(description='Reset Entire Forum', style=ButtonStyle())

Checkbox(value=False, description='Use Previous Work', indent=False)

Output()

### Step 9: Breaking The Second Sanitization Function

The developers of Sloth's Unlimited created a better sanitization function, in hopes to fix their security. This is the new function that they have written:

```
function sanitize($string) {
  $new = $string;
  do {
    $string = $new;
    $new = preg_replace("/<\/?script[^>]*>/", "", $string);
  } while (strcmp($string, $new) != 0);
  return $new;
}
```

This function will take all occurrences of ```<script>``` and replace them with an empty string.

Adjust your payload so that it will break this sanitization function. Click "Check Payload" once you have submitted a payload and would like to test it.

In [12]:
# Click the button below to check your work.
step9Complete = False

# Function to check the permissions.
def step_9():
    # Important variables that must be accessed outside of this function.
    global step9Complete, result

    with output9:
        output9.clear_output()
        if (prev_work_9.value == 0):
            display(HTML("<span>Accessing all topics on your forum. Please wait. This will take more time than the previous topics... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

        elif (prev_work_9.value == 1):
            display(HTML("<span>Checking your previous response. Please wait... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Before checking the step, see what the value of "Use Previous Work" is.
    check_prev_work = prev_work_9.value

    # This will be appended to the result subprocess. If check_prev_work is True, then just add one.
    prev_work = 0
    
    if (check_prev_work or runAllSteps):
        prev_work += 1

    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "export PATH=/home/USERNAME_GOES_HERE/.nvm/versions/node/v18.20.4/bin:$PATH; /home/.checker/section_4.py 9 ' + str(prev_work) + '"', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: green;'>Success! The Eagles page was automatically created. Stealing the admin's cookie was successful.</span>"))
            step9Complete = True

            # Automatically check the checkbox so that students don't accidentally run this step later and lose their progress.
            prev_work_9.value = True

    elif (result.returncode == 0):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>Every page on the forum was accessed, but Eagles wasn't created on the home page. This means that the admin's cookie was not stolen successfully. Try again.</span>"))
            step9Complete = False
    
    elif (result.returncode == 2):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>An error occurred when checking this step. Please contact your professor or TA.</span>"))
            step9Complete = False

    elif (result.returncode == 3):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>It appears this step wasn't attempted. Please run this step without \"Use Previous Work\" checked.</span>"))
            step9Complete = False

def check_step_9(b):
    if (warn_student()):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_9()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("9", result.returncode)

def reset_forum(b):
    subprocess.run("ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server /home/.checker/reset.sh", shell=True)
    with output9:
        output9.clear_output()
        display(HTML("<span style='color: black;'>Your forum was reset. All new messages have been deleted.</span>"))

        # Auto-save.
        if (not runAllSteps):
            trigger_save("9", result.returncode)

# Creating the buttons.
button = widgets.Button(description="Check Payload")
reset_button = widgets.Button(description="Reset Entire Forum")

# Creating a checkbox for "Use Previous Work".
prev_work_9 = widgets.Checkbox(value=False,
    description='Use Previous Work',
    disabled=False,
    indent=False)

# Creating an output area.
output9 = widgets.Output()

# Run the commands on click.
button.on_click(check_step_9)
reset_button.on_click(reset_forum)

# Display the output.
display(button, reset_button, prev_work_9, output9)

Button(description='Check Payload', style=ButtonStyle())

Button(description='Reset Entire Forum', style=ButtonStyle())

Checkbox(value=False, description='Use Previous Work', indent=False)

Output()

### Step 10: Breaking The Third Sanitization Function (OPTIONAL)

Finally, the developers have decided to use one more sanitization function in hopes to prevent all vulnerabilities involving the ```<script>``` tag. This is what they are going to use:

```
function sanitize($string) {
  $new = $string;
  do {
    $string = $new;
    $new = preg_replace("/<[^>]+?>/", "", $string);
  } while (strcmp($string, $new) != 0);
  return $new;
}
```

<strong>Since this is an optional step</strong>, you are encouraged to think about what this function does, and how it can be broken. View the differences between this step and the previous step, and try to decipher these differences.

Adjust your payload so that it will break this sanitization function.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> If you're unfamiliar with regular expressions, a very useful website to understand the regex pattern is <a href="https://regex101.com">Regex101</a>. You may access this website, and click on "Substitution" on the left side of the website. Copy/Paste the regex pattern from this step, then copy/paste a payload into the Test String field of the website. At the bottom of the site, you can view the output of the substitution, and see what your sanitization function is returning.

In [13]:
# Click the button below to check your work.
step10Complete = False

# Function to check the permissions.
def step_10():
    # Important variables that must be accessed outside of this function.
    global step10Complete, result

    with output10:
        output10.clear_output()
        if (prev_work_10.value == 0):
            display(HTML("<span>Accessing all topics on your forum. Please wait. This will take more time than the previous topics... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

        elif (prev_work_10.value == 1):
            display(HTML("<span>Checking your previous response. Please wait... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Before checking the step, see what the value of "Use Previous Work" is.
    check_prev_work = prev_work_10.value

    # This will be appended to the result subprocess. If check_prev_work is True, then just add one.
    prev_work = 0
    
    if (check_prev_work or runAllSteps):
        prev_work += 1

    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "export PATH=/home/USERNAME_GOES_HERE/.nvm/versions/node/v18.20.4/bin:$PATH; /home/.checker/section_4.py 10 ' + str(prev_work) + '"', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: green;'>Success! The Eagles page was automatically created. Stealing the admin's cookie was successful.</span>"))
            step10Complete = True

            # Automatically check the checkbox so that students don't accidentally run this step later and lose their progress.
            prev_work_10.value = True

    elif (result.returncode == 0):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>Every page on the forum was accessed, but Eagles wasn't created on the home page. This means that the admin's cookie was not stolen successfully. Try again.</span>"))
            step10Complete = False
    
    elif (result.returncode == 2):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>An error occurred when checking this step. Please contact your professor or TA.</span>"))
            step10Complete = False

    elif (result.returncode == 3):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>It appears this step wasn't attempted. Please run this step without \"Use Previous Work\" checked.</span>"))
            step10Complete = False

def check_step_10(b):
    if (warn_student()):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_10()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("10", result.returncode)

def reset_forum(b):
    subprocess.run("ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server /home/.checker/reset.sh", shell=True)
    with output10:
        output10.clear_output()
        display(HTML("<span style='color: black;'>Your forum was reset. All new messages have been deleted.</span>"))

    # Auto-save.
    if (not runAllSteps):
        trigger_save("10", result.returncode)

# Creating the buttons.
button = widgets.Button(description="Check Payload")
reset_button = widgets.Button(description="Reset Entire Forum")

# Creating a checkbox for "Use Previous Work".
prev_work_10 = widgets.Checkbox(value=False,
    description='Use Previous Work',
    disabled=False,
    indent=False)

# Creating an output area.
output10 = widgets.Output()

# Run the commands on click.
button.on_click(check_step_10)
reset_button.on_click(reset_forum)

# Display the output.
display(button, reset_button, prev_work_10, output10)

Button(description='Check Payload', style=ButtonStyle())

Button(description='Reset Entire Forum', style=ButtonStyle())

Checkbox(value=False, description='Use Previous Work', indent=False)

Output()

### Step 11: Using PHP's Built-In ```htmlspecialchars()``` Function

From the previous few steps, you should be able to observe that creating an effective sanitization function is difficult, and should be avoided. Fortunately, PHP contains a built-in function that is used for escaping dangerous characters, named ```htmlspecialchars()```. This function takes a string for an input, and replaces all unsafe characters with escape characters to prevent cross-site scripting attacks. 

You may read the ```htmlspecialchars()``` documentation <a href="https://www.php.net/manual/en/function.htmlspecialchars.php">here</a>.

<strong>While you worked throughout this topic</strong>, the notebook was automatically uncommenting and commenting out the sanitization functions that you saw above. These functions are all located in a PHP file named ```sanitize.php```, which every message gets sent through as they're rendered on the website.

Open ```/var/www/html/sanitize.php```, and go to the bottom of the file. You should view a function that looks like this:

```
/*** Your Routine Here ***/
/**
function sanitize($string) {

}
*/
/*************************/
```

This function is currently commented out. You may leave it either commented or uncommented. The "Check Payload" button at the bottom will automatically uncomment it if you forgot to do it yourself. After uncommenting, it should appear as this:

```
/*** Your Routine Here ***/
function sanitize($string) {

}
/*************************/
```

Create a sanitization function using ```htmlspecialchars()```. Then, using a payload that you previously used from Steps 7-9, test out your payload, then click "Check Payload". The admin will access every page on the forum. If the Eagles category doesn't get created, then you pass the step.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span> 
- The first function you broke from Step 7 provides a very close function to what you need to write. Consider using this payload, then call ```htmlspecialchars()``` onto the string.
- One of your sanitization routines is probably already uncommented. You may leave it uncommented, as <u>the notebook will automatically comment it out for you.</u>

In [14]:
# Click the button below to check your work.
step11Complete = False

# Function to check the permissions.
def step_11():
    # Important variables that must be accessed outside of this function.
    global step11Complete, result

    with output11:
        output11.clear_output()
        if (prev_work_11.value == 0):
            display(HTML("<span>Accessing all topics on your forum. Please wait. This will take more time than the previous topics... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

        elif (prev_work_11.value == 1):
            display(HTML("<span>Checking your previous response. Please wait... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Before checking the step, see what the value of "Use Previous Work" is.
    check_prev_work = prev_work_11.value

    # This will be appended to the result subprocess. If check_prev_work is True, then just add one.
    prev_work = 0
    
    if (check_prev_work or runAllSteps):
        prev_work += 1

    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "export PATH=/home/USERNAME_GOES_HERE/.nvm/versions/node/v18.20.4/bin:$PATH; /home/.checker/section_4.py 11 ' + str(prev_work) + '"', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: green;'>Success! The Eagles page was not created and you used htmlspecialchars.</span>"))
            step11Complete = True
    
    elif (result.returncode == 2):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>An error occurred when checking this step. sanitize.php cannot be found. Please contact your professor or TA.</span>"))
            step11Complete = False

    elif (result.returncode == 3):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>It appears this step wasn't attempted. Please run this step without \"Use Previous Work\" checked.</span>"))
            step11Complete = False

    elif (result.returncode == 4):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>Either Eagles was still created, or you did not attempt to use htmlspecialchars in sanitize.php yet. If you are already using htmlspecialchars in your payload, try resetting the forum, post a previously working payload, then try again.</span>"))
            step11Complete = False

    elif (result.returncode == 5):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>You have a payload, but you did not use htmlspecialchars in your sanitize.php file.</span>"))
            step11Complete = False

    elif (result.returncode == 0):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>You either do not have a payload on the forum, or you did not use htmlspecialchars in sanitize.php. Make sure you post your payload AFTER you create your sanitization function. Try again.</span>"))
            step11Complete = False

def check_step_11(b):
    if (warn_student()):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_11()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("11", result.returncode)

def reset_forum(b):
    subprocess.run("ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server /home/.checker/reset.sh", shell=True)
    with output11:
        output11.clear_output()
        display(HTML("<span style='color: black;'>Your forum was reset. All new messages have been deleted.</span>"))

    # Auto-save.
    if (not runAllSteps):
        trigger_save("11", result.returncode)

# Creating the buttons.
button = widgets.Button(description="Check Payload")
reset_button = widgets.Button(description="Reset Entire Forum")

# Creating a checkbox for "Use Previous Work".
prev_work_11 = widgets.Checkbox(value=False,
    description='Use Previous Work',
    disabled=False,
    indent=False)

# Creating an output area.
output11 = widgets.Output()

# Run the commands on click.
button.on_click(check_step_11)
reset_button.on_click(reset_forum)

# Display the output.
display(button, reset_button, prev_work_11, output11)

Button(description='Check Payload', style=ButtonStyle())

Button(description='Reset Entire Forum', style=ButtonStyle())

Checkbox(value=False, description='Use Previous Work', indent=False)

Output()

### Step 12: Writing Memos

Inside of ```/lab```, create two files named ```attack-memo.txt``` and ```fix-memo.txt```.

<strong>```attack-memo.txt```</strong>: Suppose that you are King Teshwan, the hacker for this website. Write a memo to Lord Dingwall, the owner of this website, explaining the attack that you made, describing how you broke each of the sanitization functions.

<strong>```fix-memo.txt```</strong>: Suppose that you are Lord Dingwall, the owner of this website. Write a memo to King Teshwan, the hacker for the website, explaining how you fixed the attacks that were made against the website.

For each of these memos, write no less than one paragraph.

Click the button below to save your work.

In [15]:
# Click the button below to check your work.
import base64

step12Complete = False

# Function to save the short answer.
def step_12():
    # Important variables that must be accessed outside of this function.
    global step12Complete, result

    # Loading, in case saving is slow.
    with output12:
        output12.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    if (userInput12_1.value == "" or userInput12_2.value == ""):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>You did not type a response for one of your memos.</span>"))
            step12Complete = False

    else:
        # Getting the student's response:
        user_input = userInput12_1.value + "\n\n" + userInput12_2.value
        
        # Build the SSH command to write to the file using 'cat' reading from stdin.
        ssh_command = [
            "ssh",
            "-i", "/home/USERNAME_GOES_HERE/.ssh/merge_key",
            "USERNAME_GOES_HERE@server",
            "cat > /home/.checker/responses/step_12_answer.txt"
        ]
        
        # Pass safe_value as input so that no shell quoting is needed.
        result = subprocess.run(ssh_command, input=user_input, text=True)
        
        if result.returncode == 1:
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: red;'>There was an error saving your response.</span>"))
                step12Complete = False
                
        elif result.returncode == 0:
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: green;'>Your response was saved.</span>"))
                step12Complete = True

def check_step_12(b):
    if warn_student():
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_12()
        if not runAllSteps:
            safe_value = f"{userInput12_1.value} {userInput12_2.value}"
            user_input_quoted = shlex.quote(safe_value)
            trigger_save("12", 1 if result.returncode == 0 else 0, user_input_quoted)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading12 = widgets.Output()
display(loading12)
with loading12:
    loading12.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput12_1 = widgets.Textarea(
    placeholder='From Teshwan (Hacker)',
    description='To Dingwall (Owner):',
    layout=widgets.Layout(width='90%', height='150px', margin='10px'),
    style={'description_width': 'initial'}
)

userInput12_2 = widgets.Textarea(
    placeholder='From Dingwall (Owner)',
    description='To Teshwan (Hacker):',
    layout=widgets.Layout(width='90%', height='150px', margin='10px'),
    style={'description_width': 'initial'}
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_12_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Process the output, then separate them.
if (result != ""):
    answers = (result.stdout).split("\n\n")
    if (len(answers) == 2):
        userInput12_1.value = answers[0]
        userInput12_2.value = (answers[1])[:-1]

    else:
        userInput12_1.value = ""
        userInput12_2.value = ""

def on_input_change(change):
    # Replace backticks with single quotes in the new value.
    new_val = change['new'].replace("`", "'")
    # Only update if there's a change (to avoid unnecessary recursion).
    if new_val != change['new']:
        change['owner'].value = new_val 

userInput12_1.observe(on_input_change, names='value')
userInput12_2.observe(on_input_change, names='value')

# After the student's response was loaded, clear the output.
loading12.clear_output()

# Creating the feedback output area.
output12 = widgets.Output()

# Creating the button.
button = widgets.Button(description="Save Response")

# Run the command on click.
button.on_click(check_step_12)

# Display the output.
display(userInput12_1, userInput12_2, button, output12)

Output()

Textarea(value='asdffdsa \' fdsa \'  \' \' "', description='To Dingwall (Owner):', layout=Layout(height='150px…

Textarea(value="fdsf dsaf ds ' fsd ' '", description='To Teshwan (Hacker):', layout=Layout(height='150px', mar…

Button(description='Save Response', style=ButtonStyle())

Output()

## <strong>Grading</strong>

To check your overall grade, click on the button below.

<strong>This is a specific note for the XSS lab:</strong> When running this auto-grader, any step that has a "Use Previous Work" checkbox next to it will use your previous response for that step. This means that in order for the auto-grader to count something as "correct", you had to have run it previously. In other labs, unchecked steps would be marked as correct if you did them. In this lab, you must have it done first before the auto-grader will mark it as correct.

In [16]:
# Click the button below to check your overall grade.
steps_to_check = [step_1, step_2, step_3, step_4, step_5, step_6, step_7, step_8, step_9, step_10, step_11, step_12]   

# Function to calculate grade after refreshing the cell
def calculate_grade(b):
    # To not auto-save at each step.
    global runAllSteps
    runAllSteps = True

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML("<span>Testing all steps. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Required for checking the boolean values.
    for func in steps_to_check:
        func()  # Call each function in order.

    # Assuming steps are updated above this cell in some way
    steps = [step1Complete, step2Complete, step3Complete, step4Complete, step5Complete, step6Complete, step7Complete, step8Complete, step9Complete, step10Complete, step11Complete, step12Complete]
    output = ""
    stepsCorrect = 0
    numOfSteps = len(steps)

    for i in range(numOfSteps):
        if (i == 9):
            stepsCorrect += 1
            output += "<div style='color: orange;'>Step 10 is optional.</div>"

        else:
            if steps[i]:
                stepsCorrect += 1
                output += "<div style='color: green;'>Step " + str(i + 1) + " is complete.</div>"
            else:
                output += "<div style='color: red;'>Step " + str(i + 1) + " is incomplete.</div>"

    output += "<div style='color: black;'>You have " + str(stepsCorrect) + " out of " + str(numOfSteps) + " steps completed.</div>"

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML(output))

    # Allows auto-saving to work on individual steps again.
    runAllSteps = False
    
# Create a button to refresh the cell and another to calculate grade.
grade_button = widgets.Button(description="Calculate Grade")

# Link buttons to functions.
grade_button.on_click(calculate_grade)

# Output area.
gradeOutput = widgets.Output()

# Display the buttons and output.
display(grade_button, gradeOutput)

Button(description='Calculate Grade', style=ButtonStyle())

Output()

### Stopping the Lab

Once you are done with the lab, click on the "Stop Lab" button below. <strong>This will delete your materialization, which will delete all of the lab's resources.</strong> Your progress is saved automatically in ```saves/``` within the sidebar of your XDC. You may load this lab in the future by clicking "Load Lab" at the top.

In [17]:
# Click the button below to stop the experiment.
def stoplab(button):
    # Check to make sure that the student wants to confirm ending the lab.
    if (confirm.value == False):
        with stop_output:
            stop_output.clear_output()
            display(HTML("<newline><span style='color: red;'>Please confirm that you wish to end the lab.</span>"))

    else:
        # Defining the lab name.
        labname = "xss"
    
        # Writing the information to an empty field below the button.
        with stop_output:
            stop_output.clear_output()
            
            display(HTML("<span>Stopping the " + labname + " lab. This will take a minute to process. Please wait.</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            stopexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /share/stopexp ' + labname + 'jup"', capture_output=True, text=True, shell=True)
            stop_output.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(stopexp.stdout)
    
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been ended.</strong></span>"))

# Creating the button.
stopButton = widgets.Button(description="Stop Lab")

# Create a confirmation check.
confirm = widgets.Checkbox(
    value=False,
    description='Confirm',
    disabled=False,
    indent=False
)

# Creating an output area.
stop_output = widgets.Output()

# Run the command on click.
stopButton.on_click(stoplab)

# Display the output.
display(confirm, stopButton, stop_output)

Checkbox(value=False, description='Confirm', indent=False)

Button(description='Stop Lab', style=ButtonStyle())

Output()